In [2]:
!pip3 install torch torchvision

distutils: c:\users\hanitha\anaconda3\Include\UNKNOWN
sysconfig: c:\users\hanitha\anaconda3\Include
user = False
home = None
root = None
prefix = None
distutils: c:\users\hanitha\anaconda3\Include\UNKNOWN
sysconfig: c:\users\hanitha\anaconda3\Include
user = False
home = None
root = None
prefix = None
You should consider upgrading via the 'c:\users\hanitha\anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install -qq transformers

In [4]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
import pandas as pd

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

In [5]:
df = pd.read_excel('bert_sample.xlsx')
df.head()

,ITEM_NAME,CATEGORY_ID
0,CALIBRACION TRANSDUCER 75 nm,CAPITAL ASSEMBLY
1,for pusher whskey,CAPITAL ASSEMBLY
2,Stat 40B Press Head Cup to Carrier from Stati...,CAPITAL ASSEMBLY
3,TRANSD. Cable (4145097103) scrw,CAPITAL ASSEMBLY
4,"ZT200 7,5BAR,13BAR60HZ NUMERO DE SERIE: AIF09...",CAPITAL ASSEMBLY


In [6]:
df['CATEGORY_ID'].value_counts()

CUTTING TOOLS        5000
CAPITAL ASSEMBLY     2000
CHEMICALS            2000
LOGISTICS SERVICE    1000
Name: CATEGORY_ID, dtype: int64

## Encoding the Labels

In [7]:
possible_labels = df.CATEGORY_ID.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'CAPITAL ASSEMBLY': 0,
 'CHEMICALS': 1,
 'CUTTING TOOLS': 2,
 'LOGISTICS SERVICE': 3}

In [8]:
df['label'] = df.CATEGORY_ID.replace(label_dict)

In [9]:
df.head(5)

,ITEM_NAME,CATEGORY_ID,label
0,CALIBRACION TRANSDUCER 75 nm,CAPITAL ASSEMBLY,0
1,for pusher whskey,CAPITAL ASSEMBLY,0
2,Stat 40B Press Head Cup to Carrier from Stati...,CAPITAL ASSEMBLY,0
3,TRANSD. Cable (4145097103) scrw,CAPITAL ASSEMBLY,0
4,"ZT200 7,5BAR,13BAR60HZ NUMERO DE SERIE: AIF09...",CAPITAL ASSEMBLY,0


## Data visualization

In [ ]:
sns.countplot(df.CATEGORY_ID)
plt.xlabel('CATEGORY_ID');

C:\Users\hanitha\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


## Train and Validation Split

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['CATEGORY_ID', 'label', 'data_type']).count()

ITEM_NAME
CATEGORY_ID       label data_type           
CAPITAL ASSEMBLY  0     train           1700
                        val              300
CHEMICALS         1     train           1700
                        val              300
CUTTING TOOLS     2     train           4250
                        val              750
LOGISTICS SERVICE 3     train            850
                        val              150

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [20]:
#training data encoding
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].ITEM_NAME.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=140,
    truncation=True,
    return_tensors='pt'
)

In [22]:
#validation data encoding
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].ITEM_NAME.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256,
    truncation=True,
    return_tensors='pt'
)

In [23]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)


In [24]:
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [26]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [29]:
dataset_train[0]

(tensor([  101, 10250, 12322, 22648,  3258,  9099,  8566, 17119,  4293, 13221,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [30]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [31]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [32]:
batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)


In [33]:
dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [34]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [35]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

In [36]:
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

## Performance Metrics

In [55]:
from sklearn.metrics import f1_score
import numpy as np

In [38]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [54]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')
    


## Training

In [56]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [61]:
def evaluate(dataloader_val):

    model.eval()    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:        
        batch = tuple(b.to(device) for b in batch)        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            #print(outputs)
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [63]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

<div class="alert alert-block alert-danger">
<b>CAUTION:</b> DON'T run the below script, this will take roughly 10hrs to complete all the ephos.
</div>

<div class="alert alert-block alert-success">
<b>Model saved:</b> Model ephos are saved in data_location folder. The model can be callable to predict the test data 
</div>

In [65]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    
    val_f1 = f1_score_func(predictions, true_vals)
    
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.33923685290971384
SequenceClassifierOutput(loss=tensor(4.7838), logits=tensor([[ 5.4640, -2.5340, -2.9422, -0.8269],
        [-1.5088, -2.7724,  5.7866, -2.3133],
        [-1.4305, -2.7547,  5.6202, -2.2257]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0165), logits=tensor([[ 5.9019, -2.4928, -3.6145, -0.7178],
        [ 5.9060, -2.4760, -3.6642, -0.6975],
        [ 4.4029, -1.6274, -4.2652,  1.2960]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(3.6754), logits=tensor([[ 5.1879, -0.9921, -3.4426, -1.3311],
        [ 1.1444, -0.2201, -4.4520,  3.0711],
        [-2.0414, -2.7824,  6.8859, -2.5655]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(3.1407), logits=tensor([[-1.7009, -2.9568,  6.1910, -2.3832],
        [ 1.3819,  2.6240, -3.0026, -0.6074],
        [ 6.1315, -2.4749, -2.8849, -1.5245]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor

SequenceClassifierOutput(loss=tensor(0.0009), logits=tensor([[ 6.0599, -2.6113, -2.9930, -1.1505],
        [ 5.9590, -2.9071, -2.2521, -1.5027],
        [ 6.1800, -2.8179, -2.7556, -1.3755]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.9525), logits=tensor([[ 6.0587, -2.6108, -2.9163, -1.1189],
        [ 5.8975, -1.6123, -3.7585, -0.8005],
        [ 0.6554, -1.0618, -3.2273,  3.4385]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(2.6890), logits=tensor([[ 5.9722, -2.8794, -2.8711, -0.9653],
        [-2.0261,  6.0373, -2.0911, -1.1725],
        [ 6.0914, -2.6291, -3.2644, -0.9571]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0011), logits=tensor([[ 6.0700, -2.6836, -3.2368, -0.9230],
        [ 6.1170, -2.7950, -3.0739, -1.0455],
        [ 6.0976, -2.7750, -3.0155, -1.1011]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0018), logits=tensor([[ 5.8332, -2.5460, 

SequenceClassifierOutput(loss=tensor(0.0014), logits=tensor([[ 5.8476, -2.7310, -3.5526, -0.2494],
        [ 5.8891, -2.8507, -1.7307, -2.0695],
        [ 6.1989, -2.6197, -2.7590, -1.3522]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0028), logits=tensor([[ 6.0598, -2.3655, -2.6939, -1.6458],
        [ 5.4073, -2.0053, -3.6956,  0.2532],
        [ 5.8700, -2.1465, -2.8380, -1.3886]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0023), logits=tensor([[ 5.4202, -2.7480, -3.6539,  0.0702],
        [ 6.1459, -2.3831, -3.2100, -1.3162],
        [ 6.1459, -2.6554, -3.1328, -1.2244]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0790), logits=tensor([[ 3.2382,  1.8541, -3.8884, -1.0088],
        [ 6.2113, -2.6928, -2.7115, -1.5554],
        [ 6.0850, -2.8510, -2.3671, -1.5080]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0008), logits=tensor([[ 6.1979, -2.7143, 

SequenceClassifierOutput(loss=tensor(1.4240), logits=tensor([[-2.0063,  6.0384, -2.1903, -0.9654],
        [-2.2231,  6.1628, -1.9085, -1.0726],
        [-2.3176, -0.0488,  4.2025, -1.8300]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(2.8769), logits=tensor([[-2.0471,  6.2320, -2.0574, -1.1832],
        [ 5.9683, -2.6586, -3.2186, -0.8945],
        [-2.2003,  6.2098, -1.9034, -1.1376]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0020), logits=tensor([[-2.3326,  5.8130, -1.1929, -1.5106],
        [-2.6885,  5.6662, -1.0198, -1.2948],
        [-1.9356,  6.0179, -2.0896, -0.8187]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(2.5555), logits=tensor([[-2.1962,  6.2173, -1.9306, -1.1428],
        [ 4.1500, -0.5474, -1.7148, -2.0220],
        [-2.9086,  0.8828,  3.7767, -1.4550]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(1.4064), logits=tensor([[-2.8775,  5.5499, 

SequenceClassifierOutput(loss=tensor(6.3176), logits=tensor([[-1.6805, -0.3058, -3.5358,  5.3911],
        [-1.0896, -1.3496, -3.2476,  5.5282],
        [-0.9546, -1.0838, -3.3909,  5.2835]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(5.9636), logits=tensor([[-1.9132,  2.0602, -4.2203,  4.4230],
        [ 5.9212, -2.7003, -3.0339, -0.8615],
        [-0.6901, -1.3114, -3.5013,  5.4987]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.2440), logits=tensor([[-0.5799,  3.0381, -4.0390,  2.3266],
        [-1.9464,  5.4008, -2.4513,  0.1831],
        [-1.6844,  3.8010, -4.5764,  2.7687]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0270), logits=tensor([[-2.0967,  6.2479, -2.0790, -1.0787],
        [-2.1138,  6.2518, -2.0273, -1.1168],
        [-0.1235,  3.0395, -1.2267, -0.6272]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(1.1194), logits=tensor([[-1.7133,  5.4341, 

...

SequenceClassifierOutput(loss=tensor(2.4318e-05), logits=tensor([[-2.7648, -3.1926,  8.6960, -3.2285],
        [-2.9058, -3.1432,  8.7384, -3.0854],
        [-2.8566, -3.1825,  8.6116, -3.1471]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(2.3603e-05), logits=tensor([[-2.8323, -3.1274,  8.7314, -3.1945],
        [-2.9745, -3.0389,  8.7041, -3.1615],
        [-2.7838, -3.1904,  8.7051, -3.1553]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(2.4318e-05), logits=tensor([[-2.6971, -3.2335,  8.6864, -3.1952],
        [-2.7383, -3.1922,  8.7227, -3.2207],
        [-2.7921, -3.2430,  8.6869, -3.1296]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(2.3484e-05), logits=tensor([[-3.0201, -3.0393,  8.7380, -3.1500],
        [-2.9680, -3.0348,  8.6870, -3.1729],
        [-2.8276, -3.2371,  8.6828, -3.1171]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(2.3484e-05), logits=tensor(

SequenceClassifierOutput(loss=tensor(2.4795e-05), logits=tensor([[-2.7979, -3.1679,  8.6739, -3.1673],
        [-2.7849, -3.1410,  8.6750, -3.2401],
        [-2.7138, -3.1477,  8.6840, -3.2548]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(2.3643e-05), logits=tensor([[-2.8805, -3.0865,  8.7119, -3.1408],
        [-2.7949, -3.1704,  8.7137, -3.1909],
        [-2.9074, -3.1105,  8.7240, -3.1289]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(2.4358e-05), logits=tensor([[-2.8691, -3.0941,  8.6805, -3.1601],
        [-2.8364, -3.0592,  8.7003, -3.2136],
        [-2.7973, -3.2283,  8.6986, -3.0889]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(2.4517e-05), logits=tensor([[-3.0674, -3.0044,  8.6920, -3.0621],
        [-3.1100, -3.0212,  8.6796, -3.0301],
        [-3.1745, -2.9554,  8.6428, -2.9856]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(2.7616e-05), logits=tensor(

SequenceClassifierOutput(loss=tensor(3.0517e-05), logits=tensor([[-2.7931, -3.1999,  8.5727, -3.1065],
        [-2.5779, -3.2518,  8.6285, -3.3260],
        [-2.2763, -3.3982,  8.4470, -3.3309]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(4.5815e-05), logits=tensor([[-2.3331, -3.3676,  8.4694, -3.3087],
        [-2.5447, -3.3946,  8.5239, -3.1139],
        [-2.0057, -3.4632,  7.9988, -3.0767]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(8.8525e-05), logits=tensor([[-2.7976, -3.2088,  8.6150, -3.1391],
        [-1.6758, -3.0884,  7.1701, -3.1369],
        [-2.7181, -3.3417,  8.6095, -3.0710]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(2.7219e-05), logits=tensor([[-2.7107, -3.3365,  8.6067, -3.0647],
        [-2.6989, -3.3876,  8.6391, -3.0843],
        [-2.5257, -3.3343,  8.5989, -3.2634]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.8643), logits=tensor([[-3

SequenceClassifierOutput(loss=tensor(3.5524e-05), logits=tensor([[-3.2939, -2.6626,  8.4738, -2.9698],
        [-3.2319, -2.6852,  8.5282, -3.0178],
        [-3.4803, -2.4083,  8.2948, -3.0011]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0001), logits=tensor([[-1.3918, -2.7640,  7.5573, -3.9007],
        [-2.9859, -2.8427,  8.5682, -3.1788],
        [-1.9097, -3.2920,  7.4022, -2.9010]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(4.0848e-05), logits=tensor([[-2.7460, -3.1913,  8.6223, -3.2170],
        [-2.9988, -2.9469,  8.3922, -2.9139],
        [-2.1368, -3.2422,  8.0821, -3.2272]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(5.9523e-05), logits=tensor([[-1.6208, -3.1022,  7.8208, -3.4982],
        [-2.5618, -3.0251,  8.5294, -3.3562],
        [-2.4738, -3.1840,  8.3864, -3.2693]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(4.0252e-05), logits=tensor([[-2

SequenceClassifierOutput(loss=tensor(2.0502), logits=tensor([[-2.6573,  3.1432, -4.2158,  3.7785],
        [ 3.1227,  3.6801, -4.8017, -1.5581],
        [ 1.4893, -3.4528, -3.2884,  5.0029]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(2.8346), logits=tensor([[-1.2900, -2.7730, -2.9765,  6.6665],
        [ 7.2725, -2.4155, -4.1190, -1.2290],
        [-1.1910, -2.4689, -2.0611,  5.7784]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(1.9619), logits=tensor([[-2.5267, -1.3998, -2.6993,  6.4532],
        [ 1.1763,  5.4106, -5.6044, -0.4568],
        [-2.3079, -1.4701, -3.0652,  6.4153]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(2.9177), logits=tensor([[ 7.3828, -2.8194, -3.8638, -1.3697],
        [-2.1183, -2.5133, -2.4927,  7.0564],
        [-2.1578, -2.5344, -2.4170,  7.0460]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0003), logits=tensor([[-2.1151, -2.5567, 

In [66]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('data_volume/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

SequenceClassifierOutput(loss=tensor(4.7838), logits=tensor([[ 5.4640, -2.5340, -2.9422, -0.8269],
        [-1.5088, -2.7724,  5.7866, -2.3133],
        [-1.4305, -2.7547,  5.6202, -2.2257]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0165), logits=tensor([[ 5.9019, -2.4928, -3.6145, -0.7178],
        [ 5.9060, -2.4760, -3.6642, -0.6975],
        [ 4.4029, -1.6274, -4.2652,  1.2960]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(3.6754), logits=tensor([[ 5.1879, -0.9921, -3.4426, -1.3311],
        [ 1.1444, -0.2201, -4.4520,  3.0711],
        [-2.0414, -2.7824,  6.8859, -2.5655]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(3.1407), logits=tensor([[-1.7009, -2.9568,  6.1910, -2.3832],
        [ 1.3819,  2.6240, -3.0026, -0.6074],
        [ 6.1315, -2.4749, -2.8849, -1.5245]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(1.7947), logits=tensor([[ 5.6961, -2.3831, 

SequenceClassifierOutput(loss=tensor(0.0009), logits=tensor([[ 6.0599, -2.6113, -2.9930, -1.1505],
        [ 5.9590, -2.9071, -2.2521, -1.5027],
        [ 6.1800, -2.8179, -2.7556, -1.3755]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.9525), logits=tensor([[ 6.0587, -2.6108, -2.9163, -1.1189],
        [ 5.8975, -1.6123, -3.7585, -0.8005],
        [ 0.6554, -1.0618, -3.2273,  3.4385]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(2.6890), logits=tensor([[ 5.9722, -2.8794, -2.8711, -0.9653],
        [-2.0261,  6.0373, -2.0911, -1.1725],
        [ 6.0914, -2.6291, -3.2644, -0.9571]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0011), logits=tensor([[ 6.0700, -2.6836, -3.2368, -0.9230],
        [ 6.1170, -2.7950, -3.0739, -1.0455],
        [ 6.0976, -2.7750, -3.0155, -1.1011]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0018), logits=tensor([[ 5.8332, -2.5460, 

SequenceClassifierOutput(loss=tensor(0.0014), logits=tensor([[ 5.8476, -2.7310, -3.5526, -0.2494],
        [ 5.8891, -2.8507, -1.7307, -2.0695],
        [ 6.1989, -2.6197, -2.7590, -1.3522]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0028), logits=tensor([[ 6.0598, -2.3655, -2.6939, -1.6458],
        [ 5.4073, -2.0053, -3.6956,  0.2532],
        [ 5.8700, -2.1465, -2.8380, -1.3886]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0023), logits=tensor([[ 5.4202, -2.7480, -3.6539,  0.0702],
        [ 6.1459, -2.3831, -3.2100, -1.3162],
        [ 6.1459, -2.6554, -3.1328, -1.2244]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0790), logits=tensor([[ 3.2382,  1.8541, -3.8884, -1.0088],
        [ 6.2113, -2.6928, -2.7115, -1.5554],
        [ 6.0850, -2.8510, -2.3671, -1.5080]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0008), logits=tensor([[ 6.1979, -2.7143, 

SequenceClassifierOutput(loss=tensor(1.4240), logits=tensor([[-2.0063,  6.0384, -2.1903, -0.9654],
        [-2.2231,  6.1628, -1.9085, -1.0726],
        [-2.3176, -0.0488,  4.2025, -1.8300]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(2.8769), logits=tensor([[-2.0471,  6.2320, -2.0574, -1.1832],
        [ 5.9683, -2.6586, -3.2186, -0.8945],
        [-2.2003,  6.2098, -1.9034, -1.1376]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0020), logits=tensor([[-2.3326,  5.8130, -1.1929, -1.5106],
        [-2.6885,  5.6662, -1.0198, -1.2948],
        [-1.9356,  6.0179, -2.0896, -0.8187]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(2.5555), logits=tensor([[-2.1962,  6.2173, -1.9306, -1.1428],
        [ 4.1500, -0.5474, -1.7148, -2.0220],
        [-2.9086,  0.8828,  3.7767, -1.4550]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(1.4064), logits=tensor([[-2.8775,  5.5499, 

...

SequenceClassifierOutput(loss=tensor(0.0001), logits=tensor([[-2.5893, -2.9179,  7.4613, -2.7157],
        [-2.5378, -3.0083,  7.4552, -2.6258],
        [-2.5615, -2.9297,  7.2017, -2.5857]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0001), logits=tensor([[-2.4765, -2.9842,  7.4584, -2.6961],
        [-2.6283, -2.9088,  7.4387, -2.6470],
        [-2.5531, -2.9349,  7.4551, -2.6642]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0001), logits=tensor([[-2.5250, -2.9479,  7.4393, -2.7024],
        [-2.4744, -2.9683,  7.4559, -2.7117],
        [-2.5116, -3.0101,  7.4411, -2.6853]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0001), logits=tensor([[-2.6028, -2.9049,  7.5055, -2.6916],
        [-2.6045, -2.8973,  7.4296, -2.6805],
        [-2.4994, -3.0258,  7.4297, -2.6854]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0001), logits=tensor([[-2.5089, -2.9995, 

SequenceClassifierOutput(loss=tensor(0.0001), logits=tensor([[-2.5715, -2.9437,  7.4328, -2.6481],
        [-2.5373, -2.9433,  7.4194, -2.7067],
        [-2.5318, -2.8873,  7.4608, -2.7353]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0001), logits=tensor([[-2.5463, -2.9365,  7.4581, -2.6570],
        [-2.4564, -2.9984,  7.4477, -2.7120],
        [-2.6155, -2.8941,  7.4491, -2.6242]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0001), logits=tensor([[-2.5736, -2.9347,  7.4645, -2.6775],
        [-2.5425, -2.9010,  7.4825, -2.7329],
        [-2.5821, -2.9590,  7.4586, -2.6377]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0001), logits=tensor([[-2.6775, -2.8589,  7.4396, -2.5914],
        [-2.6381, -2.9143,  7.4367, -2.5951],
        [-2.7169, -2.8474,  7.4262, -2.5687]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0001), logits=tensor([[-2.6742, -2.8867, 

SequenceClassifierOutput(loss=tensor(0.0001), logits=tensor([[-2.5454, -2.8625,  7.3324, -2.7019],
        [-2.4593, -2.9693,  7.4164, -2.7930],
        [-2.3601, -3.0489,  7.3297, -2.7657]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0002), logits=tensor([[-2.3441, -2.9928,  7.3261, -2.7834],
        [-2.4201, -3.0578,  7.3392, -2.6461],
        [-2.1101, -3.0622,  7.1230, -2.7400]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0002), logits=tensor([[-2.4707, -2.8573,  7.3194, -2.7673],
        [-2.1615, -2.8417,  6.9091, -2.7770],
        [-2.4344, -2.9917,  7.3369, -2.6774]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0001), logits=tensor([[-2.4488, -2.9883,  7.3333, -2.6659],
        [-2.4726, -3.0024,  7.3707, -2.6662],
        [-2.4004, -2.9934,  7.3727, -2.7468]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0015), logits=tensor([[-2.4397, -1.9710, 

SequenceClassifierOutput(loss=tensor(0.0002), logits=tensor([[-2.6280, -2.5635,  6.8935, -2.5168],
        [-2.7096, -2.6144,  7.0676, -2.4906],
        [-2.7137, -2.4934,  6.9841, -2.5950]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0004), logits=tensor([[-2.0108, -2.8768,  7.0527, -2.8175],
        [-2.5287, -2.7252,  7.1340, -2.7078],
        [-1.7875, -2.6900,  6.0998, -2.5736]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0002), logits=tensor([[-2.2566, -2.9832,  7.2781, -2.8547],
        [-2.7196, -2.7501,  7.2511, -2.4749],
        [-2.3602, -2.8223,  7.0504, -2.6461]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0002), logits=tensor([[-2.2100, -2.9158,  6.7795, -2.2831],
        [-2.2832, -2.8670,  7.2672, -2.8549],
        [-2.4592, -2.8234,  7.1722, -2.6311]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0002), logits=tensor([[-2.3351, -2.9852, 

SequenceClassifierOutput(loss=tensor(2.3880), logits=tensor([[-1.1110, -0.2181, -3.9350,  5.1773],
        [ 5.6103, -2.6642, -2.7614, -1.1096],
        [ 2.8570, -2.6156, -4.0508,  3.4669]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(3.2989), logits=tensor([[-0.4934, -1.7860, -3.1678,  5.2385],
        [ 6.0952, -2.5264, -3.4778, -0.7533],
        [ 2.7895, -1.7657, -1.5943, -0.1815]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(1.4853), logits=tensor([[-1.0166, -1.4824, -3.1487,  5.4310],
        [ 0.5154,  4.5472, -4.6282,  0.1275],
        [-1.0201, -0.9025, -3.7273,  5.3228]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(2.4784), logits=tensor([[ 6.1848, -2.8073, -2.8858, -1.2455],
        [-0.9850, -1.8209, -2.8967,  5.6505],
        [-0.9920, -1.8774, -2.8259,  5.7089]]), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.0019), logits=tensor([[-0.9710, -1.8931, 

Class: CAPITAL ASSEMBLY                                                  
Accuracy: 270/300                                  
**Percentage: 90.00**


Class: CHEMICALS                      
Accuracy: 238/300                          
**Percentage: 79.33**


Class: CUTTING TOOLS                                               
Accuracy: 743/750                                       
**Percentage: 99.06**

Class: LOGISTICS SERVICE                                                            
Accuracy: 111/150                                                         
**Percentage: 74.0**